## Apply to raster
Applies the ML model to a raster stack of bands. Requires the ML model to have already been created (by 'Get Final Classifier') and a raster stack in exactly the same format and order as the columns used to create the classifier.

In [1]:
import rasterio
import joblib
import sklearn
import numpy as np

from ErrorML.ImageToML import *

In [2]:
np.set_printoptions(suppress=True)

In [10]:
I2ML = ImageToML()

data_for_ml = I2ML.image_to_ml_input('Stacks/2017_Stack_Masked_NewOSGB_NewNoData.tif', band_to_remove=12)

In [11]:
data_for_ml.shape

(25588029, 15)

In [12]:
(data_for_ml == -9999).any()

False

In [ ]:
# with rasterio.open(I2ML.input_filename, 'r') as input_img:
#     profile = input_img.profile

# with rasterio.open('Stacks/2016_Processed.tif', 'w', **profile) as output_img:
#     output_img.write(data_for_ml)

In [ ]:
# data_for_ml[data_for_ml == -9999] = np.nan

In [ ]:
# (data_for_ml == -9999).any()

In [ ]:
# nans = np.isnan(data_for_ml)
# any_nans = np.any(nans, axis=0)

In [ ]:
# profile['count'] = 1
# profile['dtype'] = 'int8'
# profile['nodata'] = -99
# with rasterio.open('Stacks/2016_NaNs_1.tif', 'w', **profile) as output_img:
#     output_img.write(any_nans.astype(np.int8), 1)

In [13]:
classifier = joblib.load('results_newMay2017/classifiers/2017_3class_GNBPCA.jbl')

In [14]:
%time results = classifier.predict(data_for_ml).astype(float)

CPU times: user 24.5 s, sys: 31.1 s, total: 55.6 s
Wall time: 1min 5s


In [15]:
classes_3 = [-2, -0.2, 0.2, 6.5]
classes_3_to_worst_case = {0: -2,
                           1: -0.2,
                           2: 6.5,
                           -1: np.nan}

classes_3_to_best_case = {0: -0.2,
                          1: 0,
                          2: 0.2,
                          -1: np.nan}

In [16]:
def reclassify(arr, d):
    result = np.full_like(arr, 9999)
    for from_val, to_val in d.items():
        result[arr == from_val] = to_val
    
    return result

In [17]:
(results == -1).sum()

0

In [18]:
results_best_case = reclassify(results, classes_3_to_best_case)

results_worst_case = reclassify(results, classes_3_to_worst_case)

I2ML.ml_output_to_image(results_best_case, 'results_newMay2017/output_images/2017_results_3class_best_case.tif')

I2ML.ml_output_to_image(results_worst_case, 'results_newMay2017/output_images/2017_results_3class_worst_case.tif')